In [1]:
import tensorflow as tf
import numpy as np
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
n_inputs = 28*28
n_hidden1 = 300

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')

In [4]:
he_init = tf.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden1, activation = tf.nn.relu, kernel_initializer = he_init, name='hidden1')

Instructions for updating:
Use keras.layers.dense instead.


In [5]:
def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

numpy.maximum can only be used element-wise, while numpy.max and numpy.amax can be used on particular axes, or all elements.
b = np.array([3, 6, 1])
 c = np.array([4, 2, 9])
 np.maximum(b, c)
result----->array([4, 6, 9]

In [6]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

In [7]:
def leaky_relu(z, name=None):
    return tf.maximum(0.01 * z, z, name=name)

hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
reset_graph()

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_output = 10

In [ ]:
X = tf.placeholder(tf.float32, shape=(None,n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

In [ ]:
with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, activation = leaky_relu, name='hidden1')
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=leaky_relu, name='hidden2')
    logits = tf.layers.dense(hidden2, n_output, name='output')

In [ ]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

In [ ]:
learning_rate=0.01

In [ ]:
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28)/255.0
X_test = X_test.astype(np.int32).reshape(-1, 28*28)/255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_val, X_train = X_train[:5000], X_train[5000:]
y_val, y_train = y_train[:5000], y_train[5000:]

In [ ]:
X_train.shape

(55000, 784)

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected, batch_norm
from tensorflow.contrib.framework import arg_scope

tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, 28*28), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')

with tf.name_scope('dnn'):
    he_init = tf.contrib.layers.variance_scaling_initializer()
    bn_params = {
        'is_training':is_training,
        'decay':0.9,
        'updates_collections':None,
        'scale':True
    }
    
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1', normalizer_fn = batch_norm, normalizer_params=bn_params )
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2', normalizer_fn=batch_norm, normalizer_params=bn_params)
    logits = fully_connected(hidden2, n_outputs, scope='output', activation_fn=None, normalizer_fn=batch_norm, normalizer_params=bn_params)
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy,name='loss')
    
with tf.name_scope('train'):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28)/255.0
X_test = X_test.astype(np.int32).reshape(-1, 28*28)/255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [ ]:
n_epoch=20
batch_size = 50
m,n = X_train.shape
n_batch = np.int(np.ceil(m/batch_size))

def fetch_batch():
    indices = np.random.randint(m, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epoch):
        for batch_index in range(n_batch):
            X_batch, y_batch = fetch_batch()
            sess.run(training_op, feed_dict={is_training:True, X:X_batch, y:y_batch})
        
        acc_train = accuracy.eval(feed_dict={is_training:False, X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={is_training:False, X:X_test, y:y_test})
        
        print(epoch,' training accuracy ',acc_train, ' testing accuracy ',acc_test)
        
    save_path = saver.save(sess, 'my_model_final_batch_norm.ckpt')
        

0  training accuracy  1.0  testing accuracy  0.9692
1  training accuracy  1.0  testing accuracy  0.9765
2  training accuracy  1.0  testing accuracy  0.9783
3  training accuracy  0.98  testing accuracy  0.9798
4  training accuracy  1.0  testing accuracy  0.9793
5  training accuracy  1.0  testing accuracy  0.9801
6  training accuracy  1.0  testing accuracy  0.9817
7  training accuracy  1.0  testing accuracy  0.9814
8  training accuracy  1.0  testing accuracy  0.9812
9  training accuracy  1.0  testing accuracy  0.9811


In [ ]:
import tensorflow as tf

threshold = 1.0
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_grads_and_vars = [(tf.clip_by_value(grad, -threshold, threshold),var) for grad,var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_grads_and_vars)

In [ ]:
n_epoch=20
batch_size = 50
m,n = X_train.shape
n_batch = np.int(np.ceil(m/batch_size))

def fetch_batch():
    indices = np.random.randint(m, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epoch):
        for batch_index in range(n_batch):
            X_batch, y_batch = fetch_batch()
            sess.run(training_op, feed_dict={is_training:True, X:X_batch, y:y_batch})
        
        acc_train = accuracy.eval(feed_dict={is_training:False, X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={is_training:False, X:X_test, y:y_test})
        
        print(epoch,' training accuracy ',acc_train, ' testing accuracy ',acc_test)
        
    save_path = saver.save(sess, 'my_model_final_grad_clipped.ckpt')
        

In [ ]:
initial_learning_rate = 0.1
decay_steps = 10000
decay_rate = 1/10
global_step = tf.Variable(0, trainable = False)
learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate)
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, global_step=global_step )

In [ ]:
n_epoch=20
batch_size = 50
m,n = X_train.shape
n_batch = np.int(np.ceil(m/batch_size))

def fetch_batch():
    indices = np.random.randint(m, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epoch):
        for batch_index in range(n_batch):
            X_batch, y_batch = fetch_batch()
            sess.run(training_op, feed_dict={ X:X_batch, y:y_batch})
        
        acc_train = accuracy.eval(feed_dict={ X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={ X:X_test, y:y_test})
        
        print(epoch,' training accuracy ',acc_train, ' testing accuracy ',acc_test)
        
    save_path = saver.save(sess, 'my_model_final.ckpt')
        

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected, batch_norm
from tensorflow.contrib.framework import arg_scope

tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01
momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, 28*28), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')


with tf.name_scope('dnn'):
    he_init = tf.contrib.layers.variance_scaling_initializer()
    bn_params = {
        'is_training':is_training,
        'decay':0.9,
        'updates_collections':None,
        'scale':True
    }
    
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1', normalizer_fn = batch_norm, normalizer_params=bn_params )
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2', normalizer_fn=batch_norm, normalizer_params=bn_params)
    logits = fully_connected(hidden2, n_outputs, scope='output', activation_fn=None, normalizer_fn=batch_norm, normalizer_params=bn_params)
    
    

with tf.variable_scope('', default_name='', reuse=True):
    weights1 = tf.get_variable('hidden1/weights')
    weights2 = tf.get_variable('hidden2/weights')
    
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    base_loss = tf.reduce_mean(xentropy,name='avg_entropy')
    reg_loss = tf.reduce_sum(tf.abs(weights1)) + tf.reduce_sum(tf.abs(weights2))
    loss = tf.add(base_loss, 0.1*reg_loss, name='loss')
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate, momentum)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_epoch=20
batch_size = 50
m,n = X_train.shape
n_batch = np.int(np.ceil(m/batch_size))

def fetch_batch():
    indices = np.random.randint(m, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epoch):
        for batch_index in range(n_batch):
            X_batch, y_batch = fetch_batch()
            sess.run(training_op, feed_dict={is_training:True, X:X_batch, y:y_batch})
        
        acc_train = accuracy.eval(feed_dict={is_training:False, X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={is_training:False, X:X_test, y:y_test})
        
        print(epoch,' training accuracy ',acc_train, ' testing accuracy ',acc_test)
        
    save_path = saver.save(sess, 'my_model_final_l1_reg.ckpt')
        

In [ ]:
scale = 0.001

my_dense_layer = partial(
    tf.layers.dense, activation = tf.nn.relu, kernel_relularizer=tf.contrib.layers.l1_regularizer(scale)
)

with tf.name_scope('dnn'):
    hidden1 = my_dense_layer(X, n_hidden1, name='hidden1')
    hidden2 = my_dense_layer(hidden1, n_hidden2, name = 'hidden2')
    logits = my_dense_layer(hidden2, n_output, activation=None, name='output')
    
with tf.name_scope('loss'):
    xentropy = tf.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    base_loss = tf.reduce_mean(xentropy, 'xentropy')
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_loss, name='loss')

In [ ]:
n_epoch=20
batch_size = 50
m,n = X_train.shape
n_batch = np.int(np.ceil(m/batch_size))

def fetch_batch():
    indices = np.random.randint(m, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epoch):
        for batch_index in range(n_batch):
            X_batch, y_batch = fetch_batch()
            sess.run(training_op, feed_dict={is_training:True, X:X_batch, y:y_batch})
        
        acc_train = accuracy.eval(feed_dict={is_training:False, X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={is_training:False, X:X_test, y:y_test})
        
        print(epoch,' training accuracy ',acc_train, ' testing accuracy ',acc_test)
        
    save_path = saver.save(sess, 'my_model_final_l1_reg_final.ckpt')